# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [56]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [57]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [58]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [59]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [60]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [61]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [62]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [63]:
# Creating tables
tables_postfix = ["by_session_and_item", "by_user_and_session", "by_song_and_user"]
for i in range(0, 3):
    postfix = tables_postfix[i]
    table_name = "music_library_"+postfix
    query = "CREATE TABLE IF NOT EXISTS " + table_name + " "
    query = query + "(artist_name text, user_first_name text, gender varchar, item_in_session int, user_last_name text, length_of_song float, level varchar, user_location text, session_id int, song_title text, user_id int, "
    if i == 0:
        # For the table `music_library_by_session_and_item` both `session_id` and `item_in_session` were used as a partition key
        # to enable efficient querying of a specific song within a session + session item. This grouping of columns is unique
        # and we don't need extra clustering columns to sort the data.
        query = query + "PRIMARY KEY ((session_id, item_in_session))) " 
    elif i == 1:
        # For the table `music_library_by_user_and_session` both `user_id` and `session_id` were used as a partition key
        # to enable efficient querying of a song + artist for a specific user session. This grouping of columns is unique,
        # but besides the partition key a clustering columns over `item_in_session` was created to sort the data in ascending order.
        query = query + "PRIMARY KEY ((user_id, session_id), item_in_session)) " 
        query = query + "WITH CLUSTERING ORDER BY (item_in_session ASC)"
    elif i == 2:
        # For the table `music_library_by_song_and_user` `song_title` was used as a partition key
        # to enable efficient querying of a specific user listening to a song. The grouping of the `user_id` and `song_title` 
        # makes it possible to know the last time a specific user listened to a given song.
        query = query + "PRIMARY KEY ((song_title), user_id)) " 
    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [64]:
query = "SELECT artist_name, song_title, length_of_song FROM music_library_by_session_and_item "
query = query + "WHERE session_id = 338 AND item_in_session = 4"
try:
    result = session.execute(query)
    for row in result:
        print(row)
except Exception as e:
    print(e)   

In [65]:
file = 'event_datafile_new.csv'
tables_postfix = ["by_session_and_item", "by_user_and_session", "by_song_and_user"]

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        line[3] = int(line[3])
        line[5] = float(line[5])
        line[8] = int(line[8])
        line[10] = int(line[10])
        for i in range(0, 3):
            postfix = tables_postfix[i]
            query = "INSERT INTO music_library_" + postfix + " (artist_name, user_first_name, gender, item_in_session, user_last_name, length_of_song, level, user_location, session_id, song_title, user_id)"
            query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            session.execute(query, tuple(line))

#### Do a SELECT to verify that the data have been inserted into each table

In [66]:
query = "SELECT artist_name, item_in_session, song_title, length_of_song FROM music_library_by_session_and_item "
query = query + "WHERE session_id = 338 AND item_in_session = 4"
try:
    result = session.execute(query)
    for row in result:
        print(row)
except Exception as e:
    print(e)

Row(artist_name='Faithless', item_in_session=4, song_title='Music Matters (Mark Knight Dub)', length_of_song=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [67]:
query = "SELECT artist_name, song_title, length_of_song FROM music_library_by_session_and_item "
query = query + "WHERE session_id = 338 AND item_in_session = 4"
try:
    result = session.execute(query)
    for row in result:
        print(row)
except Exception as e:
    print(e)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', length_of_song=495.30731201171875)


In [68]:
query = "SELECT artist_name, song_title, user_first_name, user_last_name FROM music_library_by_user_and_session "
query = query + "WHERE user_id = 10 AND session_id = 182"
try:
    result = session.execute(query)
    for row in result:
        print(row)
except Exception as e:
    print(e)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


In [69]:
query = "SELECT user_first_name, user_last_name FROM music_library_by_song_and_user "
query = query + "WHERE song_title = 'All Hands Against His Own'"
try:
    result = session.execute(query)
    for row in result:
        print(row)
except Exception as e:
    print(e)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [70]:
tables_postfix = ["by_session_and_item", "by_user_and_session", "by_song_and_user"]

for i in range(0, 3):
    postfix = tables_postfix[i]
    query = "DROP TABLE music_library_"+postfix
    try:
        session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [71]:
session.shutdown()
cluster.shutdown()